In [3]:
!nvidia-smi

Fri Nov 25 06:48:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import os
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/kaggle//us-patent/train.csv')
eval_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/kaggle//us-patent/test.csv')

In [6]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [7]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [8]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [9]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [10]:
! pip install -q datasets
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

     |████████████████████████████████| 451 kB 14.8 MB/s 
     |████████████████████████████████| 115 kB 63.7 MB/s 
     |████████████████████████████████| 212 kB 69.6 MB/s 
     |████████████████████████████████| 182 kB 69.8 MB/s 
     |████████████████████████████████| 127 kB 73.1 MB/s 


In [11]:
model_nm = 'bert-base-uncased'

In [12]:
! pip install transformers
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 10.2 MB/s 
     |████████████████████████████████| 7.6 MB 34.9 MB/s 


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def tok_func(x): return tokz(x["input"])

In [14]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [15]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [101,
  3793,
  2487,
  1024,
  1037,
  22610,
  1025,
  3793,
  2475,
  1024,
  19557,
  18532,
  4765,
  1997,
  10796,
  1025,
  2019,
  2278,
  2487,
  1024,
  19557,
  18532,
  4765,
  102])

In [16]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [17]:
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [18]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [19]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
def corr(x,y): return np.corrcoef(x,y)

In [21]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [22]:
from transformers import TrainingArguments,Trainer

In [23]:
bs = 128
epochs = 10
lr = 8e-5

In [24]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=compute_metrics)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [26]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input, target, id, context, anchor. If input, target, id, context, anchor are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27354
  Num Epochs = 10
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 2140
  Number of trainable parameters = 109483009
You're using a BertTokenizerFast tokenizer. Please note that with a fast tok

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.039517,0.688818
2,No log,0.028728,0.781435
3,0.039200,0.028761,0.792994
4,0.039200,0.024956,0.805010
5,0.014800,0.024165,0.808528
6,0.014800,0.023900,0.814583
7,0.014800,0.022726,0.817698
8,0.007700,0.025183,0.815668
9,0.007700,0.023695,0.817435
10,0.004900,0.024049,0.817308


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input, target, id, context, anchor. If input, target, id, context, anchor are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input, target, id, context, anchor. If input, target, id, context, anchor are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoin

In [27]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input, target, id, context, anchor. If input, target, id, context, anchor are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 256


array([[ 0.296875  ],
       [ 0.58740234],
       [ 0.55859375],
       [ 0.21960449],
       [ 0.01411438],
       [ 0.55126953],
       [ 0.50830078],
       [-0.00177765],
       [ 0.2421875 ],
       [ 1.06054688],
       [ 0.23718262],
       [ 0.29394531],
       [ 0.81494141],
       [ 0.9765625 ],
       [ 0.79296875],
       [ 0.38134766],
       [ 0.35180664],
       [ 0.00426483],
       [ 0.47485352],
       [ 0.46826172],
       [ 0.52099609],
       [ 0.24511719],
       [ 0.33837891],
       [ 0.27709961],
       [ 0.52685547],
       [-0.00261116],
       [ 0.01127625],
       [ 0.0109024 ],
       [ 0.00110626],
       [ 0.66455078],
       [ 0.31103516],
       [ 0.00228691],
       [ 0.76904297],
       [ 0.55908203],
       [ 0.21801758],
       [ 0.2824707 ]])

In [28]:
preds = np.clip(preds, 0, 1)
preds

array([[0.296875  ],
       [0.58740234],
       [0.55859375],
       [0.21960449],
       [0.01411438],
       [0.55126953],
       [0.50830078],
       [0.        ],
       [0.2421875 ],
       [1.        ],
       [0.23718262],
       [0.29394531],
       [0.81494141],
       [0.9765625 ],
       [0.79296875],
       [0.38134766],
       [0.35180664],
       [0.00426483],
       [0.47485352],
       [0.46826172],
       [0.52099609],
       [0.24511719],
       [0.33837891],
       [0.27709961],
       [0.52685547],
       [0.        ],
       [0.01127625],
       [0.0109024 ],
       [0.00110626],
       [0.66455078],
       [0.31103516],
       [0.00228691],
       [0.76904297],
       [0.55908203],
       [0.21801758],
       [0.2824707 ]])